In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, preprocessing, linear_model, model_selection


import lightgbm as lgb
from sklearn.linear_model import Lasso
from xgboost import XGBClassifier

In [7]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
submission_csv = pd.read_csv("sample_submission.csv")

In [8]:
# view training data 
train_df.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


In [33]:
# view testing data 
test_df.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30
0,900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,...,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0
1,900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,...,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0
2,900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,...,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1
3,900003,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,...,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0
4,900004,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,...,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2


In [34]:
submission_csv.head()

,id,target
0,900000,0.5
1,900001,0.5
2,900002,0.5
3,900003,0.5
4,900004,0.5


In [35]:
encY = preprocessing.OrdinalEncoder()

In [36]:
# encY.fit(train_df[["score"]])

In [37]:
train_df["target"] = encY.fit_transform(train_df[["target"]])

In [38]:
train_df 

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0.0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1.0
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1.0
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1.0
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,1.380145,-0.038884,0.597111,0.854560,0.684301,-1.058618,1.310699,2,1,...,-1.594744,0.522019,0.833047,2.714125,1.290094,BABBCBBBED,455.033851,0,2,1.0
899996,899996,-1.369789,0.044841,0.015458,0.376565,-0.380529,-0.830815,-1.798458,4,1,...,2.413899,-0.674942,-0.412111,-0.030436,-3.144047,BBBGBBDQBE,134.703577,0,1,0.0
899997,899997,1.386201,-0.961150,0.725994,-0.132844,0.873911,-0.245339,-1.045786,0,0,...,-0.151930,-4.560773,-1.249154,1.793535,2.253696,AEBEDBBHBA,-99.536313,0,1,0.0
899998,899998,-1.590572,-0.509938,-1.715397,-0.249988,1.359933,1.650808,-0.058592,0,2,...,2.423670,2.110008,0.561271,-2.149610,1.019982,ADBAAADDAE,47.823039,1,2,0.0


In [10]:
test_df

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30
0,900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,...,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0
1,900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,...,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0
2,900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,...,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1
3,900003,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,...,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0
4,900004,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,...,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,1599995,0.640110,0.897808,-0.523956,1.563760,-0.092281,-0.610867,0.535426,0,1,...,2.604048,1.122867,0.518110,1.243837,0.575111,0.076372,BCBCEBHMCD,204.186539,0,0
699996,1599996,-0.191771,-0.035246,-0.118533,0.584750,2.126977,0.568659,-0.052663,4,3,...,3.029857,1.384682,-1.135740,2.982713,-1.511760,2.225218,BAABCADQFC,-97.694591,0,2
699997,1599997,-0.331704,-0.328845,-1.185503,1.022128,-0.483099,-0.107146,-0.968281,1,1,...,4.021273,-1.845266,1.096011,-2.734508,-4.885955,-2.248739,AAAJCBGQBA,130.622745,1,0
699998,1599998,-2.031073,-1.238398,0.964699,-1.045950,0.906064,0.634301,-0.707474,5,1,...,1.453864,-1.696606,1.018995,1.973697,-0.353068,-3.333449,BCBBCABNDE,-364.625148,0,0


In [5]:
train_df['f_27'].apply(list)

0         [A, B, A, B, D, A, D, B, A, B]
1         [A, C, A, C, C, A, D, C, E, B]
2         [A, A, A, E, A, B, C, K, A, D]
3         [B, D, B, B, A, A, C, B, C, B]
4         [B, D, B, C, B, B, C, H, F, E]
                       ...              
899995    [B, A, B, B, C, B, B, B, E, D]
899996    [B, B, B, G, B, B, D, Q, B, E]
899997    [A, E, B, E, D, B, B, H, B, A]
899998    [A, D, B, A, A, A, D, D, A, E]
899999    [B, C, A, A, C, A, D, S, C, E]
Name: f_27, Length: 900000, dtype: object

In [10]:
train_df['f_27'].apply(list).tolist()

[['A', 'B', 'A', 'B', 'D', 'A', 'D', 'B', 'A', 'B'],
 ['A', 'C', 'A', 'C', 'C', 'A', 'D', 'C', 'E', 'B'],
 ['A', 'A', 'A', 'E', 'A', 'B', 'C', 'K', 'A', 'D'],
 ['B', 'D', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B'],
 ['B', 'D', 'B', 'C', 'B', 'B', 'C', 'H', 'F', 'E'],
 ['B', 'D', 'A', 'E', 'D', 'B', 'D', 'E', 'D', 'A'],
 ['A', 'C', 'B', 'C', 'A', 'A', 'A', 'H', 'H', 'E'],
 ['B', 'B', 'B', 'D', 'B', 'B', 'E', 'P', 'D', 'B'],
 ['A', 'C', 'A', 'E', 'B', 'A', 'D', 'D', 'A', 'A'],
 ['B', 'A', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'D'],
 ['A', 'D', 'B', 'B', 'B', 'B', 'B', 'S', 'D', 'C'],
 ['B', 'E', 'B', 'B', 'B', 'A', 'D', 'T', 'E', 'A'],
 ['B', 'C', 'B', 'D', 'C', 'B', 'D', 'S', 'A', 'A'],
 ['A', 'D', 'B', 'C', 'D', 'A', 'G', 'K', 'B', 'E'],
 ['B', 'A', 'A', 'C', 'B', 'B', 'D', 'D', 'A', 'C'],
 ['B', 'B', 'B', 'B', 'F', 'A', 'A', 'E', 'C', 'B'],
 ['A', 'C', 'A', 'H', 'A', 'B', 'H', 'H', 'A', 'B'],
 ['A', 'B', 'B', 'I', 'B', 'B', 'C', 'J', 'F', 'D'],
 ['A', 'G', 'B', 'C', 'B', 'A', 'C', 'C', 'B',

In [11]:
pd.DataFrame(train_df['f_27'].apply(list).tolist())

,0,1,2,3,4,5,6,7,8,9
0,A,B,A,B,D,A,D,B,A,B
1,A,C,A,C,C,A,D,C,E,B
2,A,A,A,E,A,B,C,K,A,D
3,B,D,B,B,A,A,C,B,C,B
4,B,D,B,C,B,B,C,H,F,E
...,...,...,...,...,...,...,...,...,...,...
899995,B,A,B,B,C,B,B,B,E,D
899996,B,B,B,G,B,B,D,Q,B,E
899997,A,E,B,E,D,B,B,H,B,A
899998,A,D,B,A,A,A,D,D,A,E


In [40]:
# feature engineering

# function to create features from f27 column
def create_features(data):
    # create unique column names
    object_data_cols = [f"f_27_{i+1}" for i in range(10)]
    # create dataframe that has column names created above and seperates each letter into its own column
    object_data = pd.DataFrame(data['f_27'].apply(list).tolist(), columns=object_data_cols)
    
    # turn each letter into a number
    for feature in object_data_cols:
        object_data[feature] = object_data[feature].apply(ord) - ord('A')
    
    # concaatanete the original dataframe with the new dataframe created above
    data = pd.concat([data, object_data], 1)
    
    # create a column called unique_characters which counts the number of unique numbers in each row between
    # column f_27_0 and f_27_10
    data["unique_characters"] = data.f_27.apply(lambda s: len(set(s)))
    
    # return dataframe
    return data

In [41]:
# use function created above to engineering features for the training data
train_df = create_features(train_df)

# use function created above to engineering features for the testing data
test_df = create_features(test_df)

In [43]:
# remove id, f27, target from prodictor variables 
x_train = train_df.drop(["id", "f_27","target"],axis=1)

# make target dependet variable
y_train = train_df["target"]

# remove id, f27 from validation data
x_test = test_df.drop(["id","f_27",],axis=1)

In [44]:
# view a couple of rows from validation data
x_test.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_27_2,f_27_3,f_27_4,f_27_5,f_27_6,f_27_7,f_27_8,f_27_9,f_27_10,unique_characters
0,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,0,...,0,0,0,1,0,3,11,0,2,5
1,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,4,...,5,0,1,1,0,4,6,2,1,6
2,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,4,...,1,0,2,0,1,1,10,4,4,5
3,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,4,...,4,1,4,0,0,2,16,2,2,5
4,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,2,...,4,1,1,1,1,3,0,1,5,5


In [ ]:
# make sure training data and validation data have the same number of columns

In [45]:
len(x_test.columns)

41

In [46]:
len(x_train.columns)

41

<b>F1 Scores</b>

F1 scores is used to rate performance in binary classifcation. 
- f1 score is from 0 to 1
- 0 being the worst and 1 being the best

<b>Bagging and Boosting</b>

Bagging and boosting are both ensemble techniques. Weak learners are combined to make a stronger learner.
These methods helps improve a machine models performance as opposed to only using one model. It mini-
mizes noise, bias and variance.

Bagging decreases variance. The bagging approach divides the data into subsets that are fed into its own
model. The models are ran independently in parallel. The overall prediction is the average of the predictions
from each model.

Boosting decreases bias. The boosting models are ran sequentially. A model is created based on a subset of
the data. Incorrect predictions from each model are given higher weights. The next model tries to correct
the previous model errors. The overall prediction is the weighted average of of the predictions from each
model

<b>Boosting: Light Gradient Boosting Machine (LightGBM)</b>

LightGBM uses a gradient boosting technique. Decision trees are split leaf-wise where they have the best
fit.

In [81]:
# best score in kaggle so far and fastest to compute on my computer. Random forest and XGB take way too long
lgbR_model1 = lgb.LGBMClassifier(n_estimators = 10000,
                                  min_child_samples = 96,
                                  max_bins = 512,
                                  random_state = 46)

In [87]:
# do a 5 fold cross validatoin and calulate what the f1 scores will be using the above parametes
f1_scores_lgbrCLF = model_selection.cross_val_score(lgbR_model1, x_train, y_train, cv=5, scoring="f1")
f1_scores_lgbrCLF

array([0.95523122, 0.9564795 , 0.95598002, 0.95642472, 0.95614251])

In [88]:
# get the average of the f1 scores
np.mean(f1_scores_lgbrCLF)

0.9560515952698662

In [82]:
# fit the model to the data
lgbr_fitted = lgbR_model1.fit(x_train, y_train)

In [83]:
# after fitting the model make predictions
lgbr_predictions = lgbr_fitted.predict(x_test)

In [84]:
lgbr_submission_csv = submission_csv

In [85]:
# put predictions in kaggle submission format
lgbr_submission_csv["target"] = encY.inverse_transform(lgbr_predictions.reshape(-1,1))
lgbr_submission_csv.head()

,id,target
0,900000,1
1,900001,1
2,900002,0
3,900003,0
4,900004,1


In [86]:
# create a csv to submit to Kaggle
lgbr_submission_csv.to_csv("may_2022_tabular_lgbr_with_f27_10000_trees_submission.csv", index=False)